<a href="https://colab.research.google.com/github/Aancha2000l/Sentiment_Analysis/blob/main/ML_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install contractions
!pip install vaderSentiment
!pip install gitpython

     |████████████████████████████████| 327kB 5.2MB/s 
     |████████████████████████████████| 286kB 8.4MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85390 sha256=b6c49e47072a3217fe252ea2a70bef6ba316f1d08268caf501633a44f0792fa2
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick
     |████████████████████████████████| 133kB 5.8MB/s 
     |████████████████████████████████| 163kB 5.3MB/s 
     |████████████████████████████████| 71kB 5.4MB/s 


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

In [ ]:
#web scarpping excracted the html file
 
url ='https://inshorts.com/en/read/sports'
news_data = []
news_category =url.split("/")[-1]
data=requests.get(url)
soup = BeautifulSoup(data.content)
print(soup)

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<style>
    /* The Modal (background) */
    .modal_contact {
        display: none; /* Hidden by default */
        position: fixed; /* Stay in place */
        z-index: 8; /* Sit on top */
        left: 0;
        top: 0;
        width: 100%; /* Full width */
        height: 100%;
        overflow: auto; /* Enable scroll if needed */
        background-color: rgb(0,0,0); /* Fallback color */
        background-color: rgba(0,0,0,0.4); /* Black w/ opacity */
    }

    /* Modal Content/Box */
    .modal-content {
        background-color: #fefefe;
        margin: 15% auto;
        padding: 20px !important;
        padding-top: 0 !important;
        /* border: 1px solid #888; */
        text-align: center;
        position: relative;
        border-radius: 6px;
    }

    /* The Close Button */
    .close {
      left: 90%;
      color: #aaa;
      float: right;
      font-size: 28px;
      font-weight: bold;
    /* positio

In [ ]:
urls = ['https://inshorts.com/en/read/sports','https://inshorts.com/en/read/world','https://inshorts.com/en/read/politics']
def build_dataset(urls):
  news_data = []
  for url in urls:
    news_category =url.split("/")[-1]
    data = requests.get(url)
    soup = BeautifulSoup(data.content)
  
    news_articles = [{'news_headline': headline.find('span', attrs={"itemprop":"headline"}).string,
                      'news_article': article.find('div', attrs={"itemprop":"articleBody"}).string,
                      'news_category': news_category}
                     
                      for headline,article in zip(soup.find_all('div',class_=["news-card-title news-right-box"]),
                                                  soup.find_all('div',class_=["news-card-content news-right-box"]))]
    news_articles = news_articles[0:20]
    news_data.extend(news_articles)

  df = pd.DataFrame(news_data)
  df = df[['news_headline','news_article','news_category']]  
  return df                

In [ ]:
df = build_dataset(urls)
df.tail()

,news_headline,news_article,news_category
55,Priority is to end senseless violence: WB Guv ...,After Mamata Banerjee took oath as the CM of W...,politics
56,"BJP leaders provoking people, not ready to acc...",West Bengal CM Mamata Banerjee on Thursday sai...,politics
57,Nationwide lockdown only option left to check ...,Rajasthan Chief Minister Ashok Gehlot on Thurs...,politics
58,"Separatist leader Ashraf Sehrai, arrested unde...","Separatist leader Mohammad Ashraf Sehrai, who ...",politics
59,"President, PM Modi condole RLD chief Ajit Sing...",President Ram Nath Kovind and PM Narendra Modi...,politics


In [ ]:
#PREPROCESSING

import nltk
nltk.download('stopwords')
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')
len(stopword_list)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


177

In [ ]:
# removing html tags
def html_tag(text):
  soup =BeautifulSoup(text,"html.parser")
  new_text = soup.get_text()
  return new_text

html_tag('<html><h2> Some important info  </h2><html>')  
' Some important info '

' Some important info '

In [ ]:
#Expand Contractions
!pip install contractions
import contractions
def con(text):
  expand = contractions.fix(text)
  return expand

con("Y'all can't expand I'd think")

'you all can not expand I would think'

In [ ]:
# removed special character

import re
def remove_sp(text):
  pattern = r'[^A-Za-z0-9\s]'
  text = re.sub(pattern,'',text)
  return text


In [ ]:
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer =ToktokTokenizer()

In [ ]:
def remove_stopwords(text):
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  filtered_tokens = [token for token in tokens if token not in stopword_list]
  filtered_text = ' '.join(filtered_tokens)
  return filtered_text

In [ ]:
#applying to actual database

df.news_headline = df.news_headline.apply(lambda x:x.lower())
df.news_article = df.news_article.apply(lambda x:x.lower())

df.news_headline = df.news_headline.apply(html_tag)
df.news_article = df.news_article.apply(html_tag)
df.news_headline = df.news_headline.apply(con)
df.news_article = df.news_article.apply(con)
df.news_headline = df.news_headline.apply(remove_sp)
df.news_article = df.news_article.apply(remove_sp)
df.news_headline = df.news_headline.apply(remove_stopwords)
df.news_article = df.news_article.apply(remove_stopwords)

df.head()

,news_headline,news_article,news_category
0,prakash padukone hospitalised testing positive...,actress deepika padukones father former badmin...,sports
1,take jet witness bodies streets india slater a...,australian cricketerturnedcommentator michael ...,sports
2,video shows ambulance stopped ipl convoy ahmed...,video purportedly showing ambulance stopped al...,sports
3,csk member reveals dhoni delayed return says l...,csk captain ms dhoni informed teammates last p...,sports
4,covid19 entered ipl bubble difficult say soura...,bcci president sourav ganguly said difficult s...,sports


In [ ]:
# dataset labeling and processing
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs =  SentimentIntensityAnalyzer()
df['compound'] = df['news_headline'].apply(lambda x: vs.polarity_scores(x)['compound'])
df.head()

,news_headline,news_article,news_category,compound
0,prakash padukone hospitalised testing positive...,actress deepika padukones father former badmin...,sports,0.5574
1,take jet witness bodies streets india slater a...,australian cricketerturnedcommentator michael ...,sports,0.0000
2,video shows ambulance stopped ipl convoy ahmed...,video purportedly showing ambulance stopped al...,sports,-0.2263
3,csk member reveals dhoni delayed return says l...,csk captain ms dhoni informed teammates last p...,sports,-0.2732
4,covid19 entered ipl bubble difficult say soura...,bcci president sourav ganguly said difficult s...,sports,-0.3612


In [ ]:
# data finalization 
def predict(comp):
  comp=float(comp)
  if (comp>0):
    return 'positive'
  else:
    return 'negative'
df['type_pred'] = df['compound'].apply(predict)
df.head()

,news_headline,news_article,news_category,compound,type_pred
0,prakash padukone hospitalised testing positive...,actress deepika padukones father former badmin...,sports,0.5574,positive
1,take jet witness bodies streets india slater a...,australian cricketerturnedcommentator michael ...,sports,0.0000,negative
2,video shows ambulance stopped ipl convoy ahmed...,video purportedly showing ambulance stopped al...,sports,-0.2263,negative
3,csk member reveals dhoni delayed return says l...,csk captain ms dhoni informed teammates last p...,sports,-0.2732,negative
4,covid19 entered ipl bubble difficult say soura...,bcci president sourav ganguly said difficult s...,sports,-0.3612,negative


In [ ]:
df.to_csv('news.csv',index=False)

In [ ]:
df = pd.read_csv('/content/news.csv')
df


,news_headline,news_article,news_category,compound,type_pred
0,prakash padukone hospitalised testing positive...,actress deepika padukones father former badmin...,sports,0.5574,positive
1,take jet witness bodies streets india slater a...,australian cricketerturnedcommentator michael ...,sports,0.0000,negative
2,video shows ambulance stopped ipl convoy ahmed...,video purportedly showing ambulance stopped al...,sports,-0.2263,negative
3,csk member reveals dhoni delayed return says l...,csk captain ms dhoni informed teammates last p...,sports,-0.2732,negative
4,covid19 entered ipl bubble difficult say soura...,bcci president sourav ganguly said difficult s...,sports,-0.3612,negative
5,csk coaches michael hussey l balaji flown chen...,csk batting coach michael hussey bowling coach...,sports,0.4588,positive
6,fan spots exfootballers daughters face clouds ...,daughters death former uk footballer ashley ca...,sports,-0.1027,negative
7,I happy call ravindra jadeja jadeja bhogle cal...,reacting video wherein commentator harsha bhog...,sports,0.5719,positive
8,bangladesh cricket board share insensitive pos...,bangladesh cricket board shared post twitter 3...,sports,0.0772,positive
9,srhs wriddhiman saha dcs amit mishra test covi...,sunrisers hyderabad wicketkeeperbatsman wriddh...,sports,0.0000,negative


In [32]:
# dropping not required data and re-indexing the remaining data
df = df.drop(columns=['news_category'], axis=1)
df = df.drop(columns=['news_article'], axis=1)
df = df.set_index(np.arange(len(df)))
df

,news_headline,compound,type_pred
0,prakash padukone hospitalised testing positive...,0.5574,positive
1,take jet witness bodies streets india slater a...,0.0000,negative
2,video shows ambulance stopped ipl convoy ahmed...,-0.2263,negative
3,csk member reveals dhoni delayed return says l...,-0.2732,negative
4,covid19 entered ipl bubble difficult say soura...,-0.3612,negative
5,csk coaches michael hussey l balaji flown chen...,0.4588,positive
6,fan spots exfootballers daughters face clouds ...,-0.1027,negative
7,I happy call ravindra jadeja jadeja bhogle cal...,0.5719,positive
8,bangladesh cricket board share insensitive pos...,0.0772,positive
9,srhs wriddhiman saha dcs amit mishra test covi...,0.0000,negative


In [34]:
#Splitting the data into train and test data

from sklearn.model_selection import train_test_split
x = df.iloc[:,0].values
y = df.iloc[:,2].values
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=100)

In [35]:
# TFIDF Vectoriztion
# Creating the model using Multinomial Naive Bayes Agorithm
# Model is created using Pipeline

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model=Pipeline([('TFIDF',TfidfVectorizer()),('model',MultinomialNB())])
model.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('TFIDF',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('model',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [36]:
y_pred=model.predict(x_test)
y_pred

array(['negative', 'negative', 'negative', 'negative', 'negative',
       'negative', 'negative', 'negative', 'negative', 'negative',
       'negative', 'negative', 'negative', 'negative', 'negative'],
      dtype='<U8')

In [37]:
y_test

array(['negative', 'negative', 'negative', 'negative', 'negative',
       'negative', 'positive', 'negative', 'negative', 'negative',
       'negative', 'negative', 'positive', 'positive', 'positive'],
      dtype=object)

In [38]:
#evaluation

from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
accuracy_score(y_pred,y_test)

0.7333333333333333

In [ ]:
confusion_matrix(y_pred,y_test)

In [ ]:
print(classification_report(y_pred,y_test))

In [ ]:
#predicting for new value

text=['pic csks spreadout field defending 2 vs pollard last ball goes viral']
model.predict(text)

array(['sports'], dtype='<U8')

In [ ]:
# storing the model using joblib
 
import joblib
joblib.dump(model,'sentiment analysis')

['sentiment analysis']

In [ ]:
# Using Streamit
!pip install streamlit --quiet
!pip install pyngrok==4.1.1 --quiet
from pyngrok import ngrok

     |████████████████████████████████| 8.2MB 6.5MB/s 
     |████████████████████████████████| 112kB 58.1MB/s 
     |████████████████████████████████| 4.2MB 50.9MB/s 
     |████████████████████████████████| 81kB 10.7MB/s 
     |████████████████████████████████| 163kB 48.8MB/s 
     |████████████████████████████████| 122kB 50.3MB/s 
     |████████████████████████████████| 71kB 10.5MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.4 which is incompatible.


In [ ]:
%%writefile app.py
import streamlit as st
import sklearn
import joblib
model=joblib.load('sentiment analysis')
st.title('Sentiment Analysis ')
ip=st.text_input('Enter the text:')
op=model.predict([ip])
if st.button('Predict'):
  st.title(op[0])

Writing app.py


In [ ]:
#for getting url
!nohup streamlit run app.py &
url=ngrok.connect(port='8501')
url

nohup: appending output to 'nohup.out'
nohup: failed to run command 'streamlit': No such file or directory


NameError: ignored

In [ ]:
!pip install pipreqs

INFO: Successfully saved requirements file in /content/requirements.txt
